In [1]:
! python --version

Python 3.12.0


In [2]:
%%capture
! pip install llama-index
! pip install psycopg2-binary
! pip install SQLAlchemy
! pip install python-dotenv
! pip install llama-index-llms-bedrock
! pip install llama-index-embeddings-bedrock

In [3]:
from sqlalchemy import create_engine
import os
from llama_index.core.callbacks import (
    CallbackManager,
    LlamaDebugHandler,
)
from dotenv import load_dotenv

load_dotenv(verbose=True, dotenv_path="../../../.env")

llama_debug = LlamaDebugHandler(print_trace_on_end=True)
callback_manager = CallbackManager([llama_debug])

engine = create_engine(f"postgresql+psycopg2://postgres:{
                       os.environ["PG_VECTOR_PW"]}@localhost:5432/{os.environ["PG_VECTOR_DB"]}")

In [4]:
# Testing our connection
with engine.connect() as connection:
    cursor = connection.exec_driver_sql("SELECT * FROM people LIMIT 5")
    print(cursor.fetchall())
    
with engine.connect() as connection:
    cursor = connection.exec_driver_sql("SELECT * FROM planet LIMIT 5")
    print(cursor.fetchall())

[(1, 'Luke Skywalker', 172, 77.0, 'blond', 'fair', 'blue', '19BBY', 'male', 1, datetime.datetime(2014, 12, 9, 13, 50, 51, 644000), datetime.datetime(2014, 12, 20, 21, 17, 56, 891000), 'https://swapi.co/api/people/1/'), (2, 'C-3PO', 167, 75.0, 'n/a', 'gold', 'yellow', '112BBY', 'n/a', 1, datetime.datetime(2014, 12, 10, 15, 10, 51, 357000), datetime.datetime(2014, 12, 20, 21, 17, 50, 309000), 'https://swapi.co/api/people/2/'), (3, 'R2-D2', 96, 32.0, 'n/a', 'white, blue', 'red', '33BBY', 'n/a', 8, datetime.datetime(2014, 12, 10, 15, 11, 50, 376000), datetime.datetime(2014, 12, 20, 21, 17, 50, 311000), 'https://swapi.co/api/people/3/'), (4, 'Darth Vader', 202, 136.0, 'none', 'white', 'yellow', '41.9BBY', 'male', 1, datetime.datetime(2014, 12, 10, 15, 18, 20, 704000), datetime.datetime(2014, 12, 20, 21, 17, 50, 313000), 'https://swapi.co/api/people/4/'), (5, 'Leia Organa', 150, 49.0, 'brown', 'light', 'brown', '19BBY', 'female', 2, datetime.datetime(2014, 12, 10, 15, 20, 9, 791000), datetim

In [5]:
from llama_index.core import Settings
from llama_index.llms.bedrock import Bedrock
from llama_index.embeddings.bedrock import BedrockEmbedding

Settings.llm = Bedrock(
    model="anthropic.claude-3-sonnet-20240229-v1:0",
    aws_access_key_id=os.environ["AWS_ACCESS_KEY_ID"],
    aws_secret_access_key=os.environ["AWS_SECRET_ACCESS_KEY"],
    aws_session_token=os.environ["AWS_SESSION_TOKEN"],
    region_name=os.environ["AWS_DEFAULT_REGION"],
)

Settings.embed_model = BedrockEmbedding(
    model_name="cohere.embed-multilingual-v3",
    region_name=os.environ["AWS_DEFAULT_REGION"],
)

In [6]:
from llama_index.core import SQLDatabase

all_tables = [
    ("people", "This table contains all the people or characters of all Star Wars episodes."),
    ("planet", "This table contains all the planets of all Star Wars episodes."),]

sql_database = SQLDatabase(engine, include_tables=[table_name for table_name, _ in all_tables])

In [7]:
from llama_index.core.indices.struct_store import SQLTableRetrieverQueryEngine
from llama_index.core.objects import (
    SQLTableNodeMapping,
    ObjectIndex,
    SQLTableSchema,
)
from llama_index.core import VectorStoreIndex
from IPython.display import Markdown, display

In [8]:
table_node_mapping = SQLTableNodeMapping(sql_database)

table_schema_objs = []
for table_name, table_description in all_tables:
    table_schema_objs.append(
        SQLTableSchema(table_name=table_name, context_str=table_description))
    
print(table_schema_objs)

obj_index = ObjectIndex.from_objects(
    table_schema_objs,
    table_node_mapping,
    VectorStoreIndex,
    callback_manager=callback_manager,
)

query_engine = SQLTableRetrieverQueryEngine(
    sql_database,
    obj_index.as_retriever(similarity_top_k=2),
    # callback_manager=callback_manager,    # Uncomment this line to enable debugging of the query engine
)

def format_sql_results_as_markdown_table(sql_results, headers):
    """
    Formats the SQL results as a markdown table
    """
    if not sql_results and not headers:
        return "No results found."
    
    header_row = "| " + " | ".join(headers) + " |"
    separator_row = "| " + " | ".join(["---"] * len(headers)) + " |"

    table_rows = []
    for row in sql_results:
        table_row = "|"
        for j, _ in enumerate(headers):
            table_row += f" {str(row[j])} |"
        table_rows.append(table_row)

    markdown_table = "\n".join([header_row, separator_row] + table_rows)
    return markdown_table

response_template = """
### Question

**{number}.** **{question}**

### Answer

{response}

### Generated SQL Query
```sql
{sql}
```

### SQL Results

{sql_results}

"""

def text_to_sql(query_engine, question, number=1):
    """
    Calls the query engine with the given question and displays the response as a markdown cell
    """
    engine_response = query_engine.query(question)
    if "result" in engine_response.metadata and "col_keys" in engine_response.metadata:
        display(Markdown(response_template.format(
                number=number,
                question=question,
                response=str(engine_response),
                sql=engine_response.metadata["sql_query"],
                sql_results=format_sql_results_as_markdown_table(
                    engine_response.metadata["result"],
                    engine_response.metadata["col_keys"]),
            )))
    else:
        print("No results found.")

[SQLTableSchema(table_name='people', context_str='This table contains all the people or characters of all Star Wars episodes.'), SQLTableSchema(table_name='planet', context_str='This table contains all the planets of all Star Wars episodes.')]
**********
Trace: index_construction
**********


In [9]:
text_to_sql(query_engine, "How many tables are there in the database?", number=1)


### Question

**1.** **How many tables are there in the database?**

### Answer

Based on the SQL query and response, there are 2 tables in the database schema 'public'.

### Generated SQL Query
```sql
SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = 'public';
```

### SQL Results

| count |
| --- |
| 2 |



In [10]:
text_to_sql(query_engine, "How many of the planets are rocky?", number=2)


### Question

**2.** **How many of the planets are rocky?**

### Answer

Based on the SQL query and the result, there are 27 planets in the database that have a rocky, mountainous, or desert terrain. Therefore, the response to the input question "How many of the planets are rocky?" would be:

There are 27 rocky planets.

### Generated SQL Query
```sql
SELECT COUNT(*) 
FROM planet
WHERE terrain LIKE '%rocky%' OR terrain LIKE '%mountain%' OR terrain LIKE '%desert%';
```

### SQL Results

| count |
| --- |
| 27 |



In [11]:
question = """Give me the names, hair color of people who have blue eyes? Arrange the names in reverse alphabetical order."""
text_to_sql(query_engine, question, number=3)


### Question

**3.** **Give me the names, hair color of people who have blue eyes? Arrange the names in reverse alphabetical order.**

### Answer

Here are the names and hair colors of people with blue eyes, arranged in reverse alphabetical order:

Wilhuff Tarkin - auburn, grey hair
Tarfful - brown hair  
Ric Olié - brown hair
Qui-Gon Jinn - brown hair
Owen Lars - brown, grey hair
Mon Mothma - auburn hair
Mas Amedda - no hair
Luminara Unduli - black hair
Luke Skywalker - blond hair
Lobot - no hair
Jocasta Nu - white hair
Jek Tono Porkins - brown hair
Finis Valorum - blond hair
Cliegg Lars - brown hair
Chewbacca - brown hair
Beru Whitesun lars - brown hair
Barriss Offee - black hair
Anakin Skywalker - blond hair
Adi Gallia - no hair

### Generated SQL Query
```sql
SELECT name, hair_color
FROM people
WHERE eye_color = 'blue'
ORDER BY name DESC;
```

### SQL Results

| name | hair_color |
| --- | --- |
| Wilhuff Tarkin | auburn, grey |
| Tarfful | brown |
| Ric Olié | brown |
| Qui-Gon Jinn | brown |
| Owen Lars | brown, grey |
| Mon Mothma | auburn |
| Mas Amedda | none |
| Luminara Unduli | black |
| Luke Skywalker | blond |
| Lobot | none |
| Jocasta Nu | white |
| Jek Tono Porkins | brown |
| Finis Valorum | blond |
| Cliegg Lars | brown |
| Chewbacca | brown |
| Beru Whitesun lars | brown |
| Barriss Offee | black |
| Anakin Skywalker | blond |
| Adi Gallia | none |



In [12]:
question = """List the top 10 rocky planets in alphabetical order?"""
text_to_sql(query_engine, question, number=4)


### Question

**4.** **List the top 10 rocky planets in alphabetical order?**

### Answer

Based on the SQL query and response, here is a synthesized response to the input question "List the top 10 rocky planets in alphabetical order?":

The top 10 rocky planets in alphabetical order are:

1. Bestine IV (rocky islands, oceans)
2. Iktotch (rocky)
3. Iridonia (rocky canyons, acid pools)
4. Sullust (mountains, volcanoes, rocky deserts)

The query filtered the planet table to only include planets whose terrain description contained the word 'rocky'. It then ordered the results alphabetically by the planet name and limited the output to the first 10 results. However, the response only contained 4 planets, likely because there were fewer than 10 planets matching the 'rocky' terrain criteria in the database.

### Generated SQL Query
```sql
SELECT name, terrain 
         FROM planet
         WHERE terrain LIKE '%rocky%'
         ORDER BY name
         LIMIT 10;
```

### SQL Results

| name | terrain |
| --- | --- |
| Bestine IV | rocky islands, oceans |
| Iktotch | rocky |
| Iridonia | rocky canyons, acid pools |
| Sullust | mountains, volcanoes, rocky deserts |



In [13]:
question = """For all planets with rocky terrain, return a list of people and their eye colors."""
text_to_sql(query_engine, question, number=5)


### Question

**5.** **For all planets with rocky terrain, return a list of people and their eye colors.**

### Answer

Based on the SQL query and results, here is a possible response to the input question:

"For planets with rocky terrain, the people found on those planets and their respective eye colors are:

- Eeth Koth with brown eyes
- Jek Tono Porkins with blue eyes 
- Nien Nunb with black eyes
- Saesee Tiin with orange eyes"

The query joins the `people` and `planet` tables to find all people who are from planets that have 'rocky' in their terrain description. It selects the person's name and eye color, orders the results by name, and returns that information.

### Generated SQL Query
```sql
SELECT p.name, p.eye_color
FROM people p
JOIN planet pl ON p.planet_id = pl.id
WHERE pl.terrain LIKE '%rocky%'
ORDER BY p.name;
```

### SQL Results

| name | eye_color |
| --- | --- |
| Eeth Koth | brown |
| Jek Tono Porkins | blue |
| Nien Nunb | black |
| Saesee Tiin | orange |



In [14]:
question = """List top ten people (alphabetical order) who are from planets with a temperate climate? Also list their planet name."""
text_to_sql(query_engine, question, number=6)


### Question

**6.** **List top ten people (alphabetical order) who are from planets with a temperate climate? Also list their planet name.**

### Answer

Here are the top 10 people from planets with a temperate climate, listed in alphabetical order along with their planet name:

1. Ackbar from Mon Cala
2. Adi Gallia from Coruscant  
3. Ayla Secura from Ryloth
4. Bail Prestor Organa from Alderaan
5. Bib Fortuna from Ryloth
6. Boba Fett from Kamino
7. Cordé from Naboo
8. Darth Maul from Dathomir
9. Dormé from Naboo
10. Dud Bolt from Vulpter

### Generated SQL Query
```sql
SELECT p.name, pl.name AS planet_name
FROM people p
JOIN planet pl ON p.planet_id = pl.id
WHERE pl.climate LIKE '%temperate%'
ORDER BY p.name
LIMIT 10;
```

### SQL Results

| name | planet_name |
| --- | --- |
| Ackbar | Mon Cala |
| Adi Gallia | Coruscant |
| Ayla Secura | Ryloth |
| Bail Prestor Organa | Alderaan |
| Bib Fortuna | Ryloth |
| Boba Fett | Kamino |
| Cordé | Naboo |
| Darth Maul | Dathomir |
| Dormé | Naboo |
| Dud Bolt | Vulpter |



In [15]:
question = """What are the names and birth years of all people from planets with a population greater than 1,000,000? Also include the polulation. Top 10 only please."""
text_to_sql(query_engine, question, number=7)


### Question

**7.** **What are the names and birth years of all people from planets with a population greater than 1,000,000? Also include the polulation. Top 10 only please.**

### Answer

Based on the query results, here is a response synthesized from the top 10 results:

The people with the highest planet populations are Jocasta Nu, Finis Valorum, and Adi Gallia, all from planets with a population of 1 quadrillion. Wat Tambor is from a planet with 500 trillion inhabitants, while Poggle the Lesser's planet has a population of 100 trillion. 

Other notable individuals include Ackbar (born 41BBY) from a planet with 27 billion residents, Wilhuff Tarkin (born 64BBY) from a planet with 22 billion, Nien Nunb from a planet with 18.5 billion, the ancient Jabba Desilijic Tiure (born 600BBY) from a planet with 7 billion, and San Hill from a planet with a population of 5 billion.

### Generated SQL Query
```sql
SELECT p.name, p.birth_year, pl.population
FROM people p
JOIN planet pl ON p.planet_id = pl.id
WHERE pl.population > 1000000
ORDER BY pl.population DESC
LIMIT 10;
```

### SQL Results

| name | birth_year | population |
| --- | --- | --- |
| Jocasta Nu | None | 1000000000000 |
| Finis Valorum | 91BBY | 1000000000000 |
| Adi Gallia | None | 1000000000000 |
| Wat Tambor | None | 500000000000 |
| Poggle the Lesser | None | 100000000000 |
| Ackbar | 41BBY | 27000000000 |
| Wilhuff Tarkin | 64BBY | 22000000000 |
| Nien Nunb | None | 18500000000 |
| Jabba Desilijic Tiure | 600BBY | 7000000000 |
| San Hill | None | 5000000000 |



In [16]:
question = """What is the average height of people grouped by their planet? Please return only the top 5 results."""
text_to_sql(query_engine, question, number=8)


### Question

**8.** **What is the average height of people grouped by their planet? Please return only the top 5 results.**

### Answer

Based on the SQL query and results, here is a synthesized response to the input question:

The top 5 planets with the tallest average height of people are:

1. Quermia, with an average height of 264 cm
2. Kashyyyk, with an average height of 231 cm 
3. Kalee, with an average height of 216 cm
4. Kamino, with an average height of 208.33 cm
5. Utapau, with an average height of 206 cm

The query first joins the people and planet tables to associate each person with their home planet. It then calculates the average height grouped by planet name, orders the results by the average height in descending order, and limits the output to the top 5 rows.

### Generated SQL Query
```sql
SELECT p.name AS planet_name, AVG(pe.height) AS average_height
FROM people pe
JOIN planet p ON pe.planet_id = p.id
GROUP BY p.name
ORDER BY average_height DESC
LIMIT 5;
```

### SQL Results

| planet_name | average_height |
| --- | --- |
| Quermia | 264.0000000000000000 |
| Kashyyyk | 231.0000000000000000 |
| Kalee | 216.0000000000000000 |
| Kamino | 208.3333333333333333 |
| Utapau | 206.0000000000000000 |



In [17]:
question = """What are the names of planets and the total surface water available on each planet, grouped by planet name? Give me the results in descending order of surface water and only the top ten results. NULL surface water value is 0 percent."""
text_to_sql(query_engine, question, number=9)


### Question

**9.** **What are the names of planets and the total surface water available on each planet, grouped by planet name? Give me the results in descending order of surface water and only the top ten results. NULL surface water value is 0 percent.**

### Answer

Based on the SQL query and results, here is a synthesized response to the input question:

The top 10 planets with the highest surface water percentage are:

1. Hoth - 100% surface water
2. Kamino - 100% surface water 
3. Ojom - 100% surface water
4. Mon Cala - 100% surface water
5. Bestine IV - 98% surface water
6. Glee Anselm - 80% surface water
7. Corellia - 70% surface water
8. Rodia - 60% surface water
9. Kashyyyk - 60% surface water
10. Chandrila - 40% surface water

The query selects the planet name and surface water percentage, groups the results by planet name, orders them in descending order based on the surface water percentage value, and limits the output to the top 10 results. Planets with null surface water values are considered to have 0% surface water.

### Generated SQL Query
```sql
SELECT p.name, COALESCE(p.surface_water, '0%') AS surface_water
FROM planet p
GROUP BY p.name, p.surface_water
ORDER BY CASE
           WHEN p.surface_water LIKE '%_%' THEN CAST(REPLACE(p.surface_water, '%', '') AS FLOAT)
           ELSE 0
         END DESC
LIMIT 10;
```

### SQL Results

| name | surface_water |
| --- | --- |
| Hoth | 100 |
| Kamino | 100 |
| Ojom | 100 |
| Mon Cala | 100 |
| Bestine IV | 98 |
| Glee Anselm | 80 |
| Corellia | 70 |
| Rodia | 60 |
| Kashyyyk | 60 |
| Chandrila | 40 |



In [18]:
question = """List the names of all female people and their planet's name."""
text_to_sql(query_engine, question, number=10)


### Question

**10.** **List the names of all female people and their planet's name.**

### Answer

Based on the SQL query and results, here is a synthesized response:

The query lists the names of all female people and the name of their home planet. Some notable entries include:

- Leia Organa from Alderaan
- Padmé Amidala from Naboo 
- Rey from an unknown planet
- Shaak Ti from Shili
- Shmi Skywalker from Tatooine
- Jedi like Adi Gallia from Coruscant, Luminara Unduli from Mirial, and Ayla Secura from Ryloth
- Other characters like Captain Phasma, Cordé, Dormé, Jocasta Nu, Mon Mothma, R4-P17, Sly Moore, Taun We, and Zam Wesell from various planets including some unknowns.

The results show a diverse range of female characters from many different planets across the Star Wars universe.

### Generated SQL Query
```sql
SELECT p.name, pl.name
FROM people p
JOIN planet pl ON p.planet_id = pl.id
WHERE p.gender = 'female'
ORDER BY p.name;
```

### SQL Results

| name | name |
| --- | --- |
| Adi Gallia | Coruscant |
| Ayla Secura | Ryloth |
| Barriss Offee | Mirial |
| Beru Whitesun lars | Tatooine |
| Captain Phasma | Unknown |
| Cordé | Naboo |
| Dormé | Naboo |
| Jocasta Nu | Coruscant |
| Leia Organa | Alderaan |
| Luminara Unduli | Mirial |
| Mon Mothma | Chandrila |
| Padmé Amidala | Naboo |
| R4-P17 | Unknown |
| Rey | Unknown |
| Shaak Ti | Shili |
| Shmi Skywalker | Tatooine |
| Sly Moore | Umbara |
| Taun We | Kamino |
| Zam Wesell | Zolan |



In [19]:
question = """Liste die Namen aller weiblichen Personen und den Namen ihres Planeten auf."""
# Now query in German (List the names of all female people and their planet's name.)
text_to_sql(query_engine, question, number=11)


### Question

**11.** **Liste die Namen aller weiblichen Personen und den Namen ihres Planeten auf.**

### Answer

Die Abfrage listet die Namen aller weiblichen Personen und den Namen ihrer jeweiligen Heimatplaneten auf. Hier sind die Ergebnisse:

Adi Gallia vom Planeten Coruscant
Ayla Secura vom Planeten Ryloth  
Barriss Offee vom Planeten Mirial
Beru Whitesun lars vom Planeten Tatooine
Captain Phasma von einem unbekannten Planeten
Cordé vom Planeten Naboo
Dormé vom Planeten Naboo
Jocasta Nu vom Planeten Coruscant
Leia Organa vom Planeten Alderaan
Luminara Unduli vom Planeten Mirial
Mon Mothma vom Planeten Chandrila
Padmé Amidala vom Planeten Naboo
R4-P17 von einem unbekannten Planeten
Rey von einem unbekannten Planeten
Shaak Ti vom Planeten Shili
Shmi Skywalker vom Planeten Tatooine
Sly Moore vom Planeten Umbara
Taun We vom Planeten Kamino
Zam Wesell vom Planeten Zolan

### Generated SQL Query
```sql
SELECT p.name AS person_name, pl.name AS planet_name
FROM people p
JOIN planet pl ON p.planet_id = pl.id
WHERE p.gender = 'female'
ORDER BY p.name;
```

### SQL Results

| person_name | planet_name |
| --- | --- |
| Adi Gallia | Coruscant |
| Ayla Secura | Ryloth |
| Barriss Offee | Mirial |
| Beru Whitesun lars | Tatooine |
| Captain Phasma | Unknown |
| Cordé | Naboo |
| Dormé | Naboo |
| Jocasta Nu | Coruscant |
| Leia Organa | Alderaan |
| Luminara Unduli | Mirial |
| Mon Mothma | Chandrila |
| Padmé Amidala | Naboo |
| R4-P17 | Unknown |
| Rey | Unknown |
| Shaak Ti | Shili |
| Shmi Skywalker | Tatooine |
| Sly Moore | Umbara |
| Taun We | Kamino |
| Zam Wesell | Zolan |



In [20]:
question = """모든 여성의 이름과 그들의 행성 이름을 나열하세요."""
# Now query in Korean (List the names of all female people and their planet's name.)
text_to_sql(query_engine, question, number=12)


### Question

**12.** **모든 여성의 이름과 그들의 행성 이름을 나열하세요.**

### Answer

여성 이름과 행성 이름은 다음과 같습니다:

Adi Gallia - 코러산트
Ayla Secura - 라일로스  
Barriss Offee - 미리알
Beru Whitesun lars - 타투인
Captain Phasma - 알려지지 않음
Cordé - 나부
Dormé - 나부
Jocasta Nu - 코러산트
Leia Organa - 알데라안
Luminara Unduli - 미리알
Mon Mothma - 찬드릴라
Padmé Amidala - 나부
R4-P17 - 알려지지 않음
Rey - 알려지지 않음  
Shaak Ti - 실리
Shmi Skywalker - 타투인
Sly Moore - 움바라
Taun We - 카미노
Zam Wesell - 졸란

### Generated SQL Query
```sql
SELECT p.name AS person_name, pl.name AS planet_name
FROM people p
JOIN planet pl ON p.planet_id = pl.id
WHERE p.gender = 'female'
ORDER BY p.name;
```

### SQL Results

| person_name | planet_name |
| --- | --- |
| Adi Gallia | Coruscant |
| Ayla Secura | Ryloth |
| Barriss Offee | Mirial |
| Beru Whitesun lars | Tatooine |
| Captain Phasma | Unknown |
| Cordé | Naboo |
| Dormé | Naboo |
| Jocasta Nu | Coruscant |
| Leia Organa | Alderaan |
| Luminara Unduli | Mirial |
| Mon Mothma | Chandrila |
| Padmé Amidala | Naboo |
| R4-P17 | Unknown |
| Rey | Unknown |
| Shaak Ti | Shili |
| Shmi Skywalker | Tatooine |
| Sly Moore | Umbara |
| Taun We | Kamino |
| Zam Wesell | Zolan |



In [21]:
question = """Isalista ang mga pangalan ng lahat ng babae at ang pangalan ng kanilang planeta."""
# Now query in Filipino (List the names of all female people and their planet's name.)
text_to_sql(query_engine, question, number=13)


### Question

**13.** **Isalista ang mga pangalan ng lahat ng babae at ang pangalan ng kanilang planeta.**

### Answer

Narito ang mga pangalan ng lahat ng babae at ang kanilang planeta:

Adi Gallia - Coruscant
Ayla Secura - Ryloth  
Barriss Offee - Mirial
Beru Whitesun lars - Tatooine
Captain Phasma - Unknown
Cordé - Naboo
Dormé - Naboo
Jocasta Nu - Coruscant
Leia Organa - Alderaan
Luminara Unduli - Mirial
Mon Mothma - Chandrila
Padmé Amidala - Naboo
R4-P17 - Unknown
Rey - Unknown
Shaak Ti - Shili
Shmi Skywalker - Tatooine
Sly Moore - Umbara
Taun We - Kamino
Zam Wesell - Zolan

### Generated SQL Query
```sql
SELECT people.name, planet.name
FROM people
JOIN planet ON people.planet_id = planet.id
WHERE people.gender = 'female'
ORDER BY people.name;
```

### SQL Results

| name | name |
| --- | --- |
| Adi Gallia | Coruscant |
| Ayla Secura | Ryloth |
| Barriss Offee | Mirial |
| Beru Whitesun lars | Tatooine |
| Captain Phasma | Unknown |
| Cordé | Naboo |
| Dormé | Naboo |
| Jocasta Nu | Coruscant |
| Leia Organa | Alderaan |
| Luminara Unduli | Mirial |
| Mon Mothma | Chandrila |
| Padmé Amidala | Naboo |
| R4-P17 | Unknown |
| Rey | Unknown |
| Shaak Ti | Shili |
| Shmi Skywalker | Tatooine |
| Sly Moore | Umbara |
| Taun We | Kamino |
| Zam Wesell | Zolan |

